#9.2. 웹에서 데이터 가오는 원리

##예제1)	Html 구조 기본 개념

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>예제 페이지</title>
</head>
<body>
    <h1>안녕하세요, 웹 크롤링!</h1>
    <p>이것은 웹 페이지의 예제입니다.</p>
    <a href="https://example.com">이동하기</a>
    <div class="menu"> 클라스 적용</div>
</body>
</html>


In [ ]:
/* 모든 <p> 태그 */
p { color: blue; }
/* 클래스가 'menu'인 요소 */
.menu { font-size: 16px; }
/* id가 'header'인 요소 */
#header { background-color: gray; }


---

In [ ]:
!pip install requests

##예제2)	https://www.google.com 웹페이지 요청

In [ ]:
import requests
# 크롤링할 웹사이트 URL
url = "https://www.dongseo.ac.kr"
# GET 요청으로 웹페이지 가져오기
response = requests.get(url)
# 응답 코드 출력 (200이면 정상)
print("응답 코드:", response.status_code)
# 가져온 HTML 일부 출력 (500자)
print(response.text[:500])


##예제3)	요청 실패 예제

In [ ]:
url = "https://www.google.com/없는페이지"
response = requests.get(url)

if response.status_code == 200:
    print("요청 성공")
    print(response.text[:500])
else:
    print(f"요청 실패: {response.status_code}")


#9.4. HTML을 BeautifulSoup으로 분석하기

In [ ]:
!pip install beautifulsoup4

##예제4)	HTML 문서 분석

In [ ]:
from bs4 import BeautifulSoup
# 예제 HTML 코드
html = """
<html>
    <head><title>웹 크롤링 연습</title></head>
    <body>
        <h1>안녕하세요!</h1>
        <p class="info">이것은 첫 번째 문장입니다.</p>
        <p class="info">이것은 두 번째 문장입니다.</p>
        <a href="https://example.com">예제 링크</a>
    </body>
</html>
"""
# BeautifulSoup 객체 생성
soup = BeautifulSoup(html, 'html.parser')
# HTML 요소 추출
print("페이지 제목:", soup.title.text)  # <title> 태그 내용 가져오기
print("헤더 제목:", soup.h1.text)  # <h1> 태그 내용 가져오기
print("첫 번째 문단:", soup.p.text)  # <p> 태그 내용 가져오기
print("링크 주소:", soup.a['href'])  # <a> 태그의 href 속성 값 가져오기


##예제5)	더 많은 데이터 가져오기



In [ ]:
all_p = soup.find_all("p")  # 모든 <p> 태그 찾기

for p in all_p:
    print("문단:", p.text)

---

#9.5. 실전 예제

##예제6)	회사 홍보 웹사이트의 헤드라인 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup

# 1. 요청할 URL 설정
url = "https://www.chemstar.kr"
response = requests.get(url)

# 2. 웹 페이지 응답 확인
if response.status_code == 200:  # HTTP 요청 성공 여부 확인
    html = response.text  # 웹 페이지의 HTML 소스 가져오기
    soup = BeautifulSoup(html, 'html.parser')  # HTML 소스를 BeautifulSoup을 사용해 파싱

    # 3. 특정 태그의 내용 추출
    titles = soup.find_all('h1')  # 모든 <h1> 태그를 리스트로 가져오기
    print("사이트 제목:")
    for idx, title in enumerate(titles):  # 가져온 <h1> 태그를 순회하며 내용 출력
        print(f"{idx + 1}. {title.get_text()}")  # 태그 내용에서 텍스트만 추출하여 출력
else:
    # 요청 실패 시 상태 코드 출력
    print(f"웹 페이지를 불러오지 못했습니다. 상태 코드: {response.status_code}")

##예제7)	뉴스 웹사이트에서 기사 제목 크롤링

###(1) 라이브러리 불러오기

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv


###(2) 크롤링할 웹사이트 주소 설정

In [ ]:
url = "https://news.naver.com/section/104"

###(3) 크롤링 차단 방지를 위한 헤더 설정

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0"
}


###(4) 웹페이지 요청 (데이터 가져오기)

In [ ]:
response = requests.get(url, headers=headers)

###(5) 요청이 성공했는지 확인

In [ ]:
# 4. 요청이 성공했는지 확인
if response.status_code == 200:
    # 5. HTML을 분석할 수 있는 형태로 변환
    soup = BeautifulSoup(response.text, "html.parser")
    # 6. 뉴스 섹션(id="newsct") 찾기
    news_section = soup.find(id="newsct")
    if news_section:
        # 7. 뉴스 제목 가져오기 (strong 태그 + 클래스 "sa_text_strong")
        headlines = news_section.find_all("strong", class_="sa_text_strong")
        # 8. 뉴스 제목 출력
        print("네이버 뉴스 (세계 섹션) 헤드라인:")
        news_list = []
        for idx, headline in enumerate(headlines, start=1):
            title = headline.text.strip()
            print(f"{idx}. {title}")
            news_list.append([idx, title])  # CSV 저장을 위해 리스트에 추가
        # 9. CSV 파일로 저장
        with open("naver_news_headlines_si.csv", "w", newline="", encoding="utf-8-sig") as file:
            writer = csv.writer(file)
            writer.writerow(["번호", "뉴스 제목"])  # CSV 헤더 추가
            writer.writerows(news_list)  # 뉴스 제목 데이터 저장
        print("\n뉴스 헤드라인이 'naver_news_headlines.csv' 파일에 저장되었습니다.")
    else:
        print("❌ 뉴스 섹션을 찾을 수 없습니다.")
else:
    print(f"❌ 페이지를 불러올 수 없습니다. 상태 코드: {response.status_code}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
# 1. 크롤링할 네이버 뉴스 "세계" 섹션 URL
url = "https://news.naver.com/section/104"
# 2. 크롤링 차단 방지를 위한 헤더 설정
headers = {
    "User-Agent": "Mozilla/5.0"
}
# 3. 웹페이지 요청 (데이터 가져오기)
response = requests.get(url, headers=headers)
# 4. 요청이 성공했는지 확인
if response.status_code == 200:
    # 5. HTML을 분석할 수 있는 형태로 변환
    soup = BeautifulSoup(response.text, "html.parser")
    # 6. 뉴스 섹션(id="newsct") 찾기
    news_section = soup.find(id="newsct")
    if news_section:
        # 7. 뉴스 제목 가져오기 (strong 태그 + 클래스 "sa_text_strong")
        headlines = news_section.find_all("strong", class_="sa_text_strong")
        # 8. 뉴스 제목 출력
        print("네이버 뉴스 (세계 섹션) 헤드라인:")
        news_list = []
        for idx, headline in enumerate(headlines, start=1):
            title = headline.text.strip()
            print(f"{idx}. {title}")
            news_list.append([idx, title])  # CSV 저장을 위해 리스트에 추가
        # 9. CSV 파일로 저장
        with open("naver_news_headlines.csv", "w", newline="", encoding="utf-8-sig") as file:
            writer = csv.writer(file)
            writer.writerow(["번호", "뉴스 제목"])  # CSV 헤더 추가
            writer.writerows(news_list)  # 뉴스 제목 데이터 저장
        print("\n뉴스 헤드라인이 'naver_news_headlines.csv' 파일에 저장되었습니다.")
    else:
        print("❌ 뉴스 섹션을 찾을 수 없습니다.")
else:
    print(f"❌ 페이지를 불러올 수 없습니다. 상태 코드: {response.status_code}")


##예제8)	네이버 뉴스 제목에서 단어 빈도수 분석

In [ ]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib


In [ ]:
print(news_list)

In [ ]:
from collections import Counter

#뉴스 제목 리스트에서 텍스트만 추출
news_titles = []
for item in news_list:
    news_titles.append(item[1])  # 뉴스 제목(두 번째 요소)만 저장

#모든 뉴스 제목을 하나의 문자열로 합친 후 단어 리스트 생성
all_words = " ".join(news_titles).split()

# 3️⃣ 단어 빈도수 계산
word_counts = Counter(all_words)

# 4️⃣ 가장 많이 등장한 단어 TOP 5 출력
print("가장 많이 등장한 단어 TOP 5:")
for word, count in word_counts.most_common(5):
    print(f"{word}: {count}회")


In [ ]:
!pip install wordcloud

In [ ]:
!apt-get update
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

##예제8) 최종코드

In [ ]:
#1.필요한 라이브러리 불러오기
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import csv

#2.한글폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

#3.CSV 파일에서 뉴스 제목 불러오기
news_titles = []
with open("naver_news_headlines.csv", "r", encoding="utf-8-sig") as file:
    reader = csv.reader(file)
    next(reader)  # 첫 번째 줄 (헤더) 건너뛰기
    for row in reader:
        news_titles.append(row[1])  # 뉴스 제목(두 번째 요소)만 리스트에 추가

#4.모든 뉴스 제목을 하나의 문자열로 합친 후 단어 리스트 생성
all_words = " ".join(news_titles).split()

#5.단어 빈도수 계산
word_counts = Counter(all_words)

#6.가장 많이 등장한 단어 TOP 5 출력
print("가장 많이 등장한 단어 TOP 5:")
for word, count in word_counts.most_common(5):
    print(f"{word}: {count}회")

#7.워드 클라우드 생성
wordcloud = WordCloud(font_path=font_path, background_color="white", width=800, height=400).generate_from_frequencies(word_counts)

#8.워드 클라우드 출력
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")  # 축 제거
plt.show()